In [1]:
import pandas as pd
import random
import re
import csv

# Charger le fichier CSV avec pandas
data_path = "./../dataset/communes-departement-region.csv"
villes_df = pd.read_csv(data_path)

# Extraire les noms des communes uniques
villes = villes_df['nom_commune_complet'].dropna().unique().tolist()

# Templates de phrases
templates = [
    "Prévois un déplacement de <VILLE_DEPART> vers <VILLE_ARRIVEE>, avec une escale possible à <VILLE_ETAPE>.",
    "Quel est l’itinéraire optimal pour partir de <VILLE_DEPART> et atteindre <VILLE_ARRIVEE> ?",
    "En quittant <VILLE_DEPART>, comment se rendre à <VILLE_ARRIVEE> tout en passant par <VILLE_ETAPE> ?",
    "Planifie un trajet pour relier <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "À partir de <VILLE_DEPART>, direction <VILLE_ARRIVEE> avec un éventuel arrêt à <VILLE_ETAPE>.",
    "Je cherche la meilleure route pour voyager depuis <VILLE_DEPART> jusqu’à <VILLE_ARRIVEE>.",
    "Comment organiser un voyage de <VILLE_DEPART> vers <VILLE_ARRIVEE>, avec une pause à <VILLE_ETAPE> ?",
    "Peux-tu m'indiquer comment aller de <VILLE_DEPART> jusqu'à <VILLE_ARRIVEE> ?",
    "Pour aller de <VILLE_DEPART> à <VILLE_ARRIVEE>, quel est l’itinéraire recommandé ?",
    "Quels sont les trajets possibles pour relier <VILLE_DEPART> à <VILLE_ARRIVEE> ?",
    "En partant de <VILLE_DEPART>, quelle serait la meilleure façon d’arriver à <VILLE_ARRIVEE> ?",
    "Trajet souhaité depuis <VILLE_DEPART> en direction de <VILLE_ARRIVEE> avec escale à <VILLE_ETAPE>.",
    "Déterminer un itinéraire allant de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Pour un trajet depuis <VILLE_DEPART> vers <VILLE_ARRIVEE>, quelles sont les options recommandées ?",
    "Je veux découvrir la meilleure route de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Est-il possible de faire un trajet de <VILLE_DEPART> jusqu'à <VILLE_ARRIVEE> avec un arrêt à <VILLE_ETAPE> ?",
    "Quel chemin suivre pour aller de <VILLE_DEPART> à <VILLE_ARRIVEE> ?",
    "En voyageant de <VILLE_DEPART> à <VILLE_ARRIVEE>, comment ajouter une escale à <VILLE_ETAPE> ?",
    "Quelles sont les étapes suggérées pour un voyage entre <VILLE_DEPART> et <VILLE_ARRIVEE> ?",
    "Je veux des informations sur un parcours de <VILLE_DEPART> à <VILLE_ARRIVEE>, avec un arrêt possible à <VILLE_ETAPE>."
]


def generate_phrase_and_labels(templates, villes):
    template = random.choice(templates)

    # Sélection de villes de départ, arrivée, et étape
    ville_depart = random.choice(villes)
    ville_arrivee = random.choice(villes)
    while ville_arrivee == ville_depart:
        ville_arrivee = random.choice(villes)

    ville_etape = None
    if "<VILLE_ETAPE>" in template:
        ville_etape = random.choice(villes)
        while ville_etape == ville_depart or ville_etape == ville_arrivee:
            ville_etape = random.choice(villes)

    # Création des tokens et des labels
    tokens = []
    labels = []

    # Utilisation de regex pour détecter et remplacer les balises
    def replace_and_label(match, ville, label_prefix):
        ville_tokens = ville.split()
        tokens.extend(ville_tokens)
        labels.extend([f"B-{label_prefix}"] + [f"I-{label_prefix}"] * (len(ville_tokens) - 1))

    # Remplacement des balises par les noms de villes et attribution des labels
    pos = 0
    for match in re.finditer(r"<(VILLE_DEPART|VILLE_ARRIVEE|VILLE_ETAPE)>", template):
        # Ajouter les mots entre les balises en tant que tokens normaux avec le label 'O'
        if match.start() > pos:
            words_between = template[pos:match.start()].split()
            tokens.extend(words_between)
            labels.extend(["O"] * len(words_between))

        # Identification de la balise et application du remplacement
        if match.group(0) == "<VILLE_DEPART>":
            replace_and_label(match, ville_depart, "depart")
        elif match.group(0) == "<VILLE_ARRIVEE>":
            replace_and_label(match, ville_arrivee, "arrive")
        elif match.group(0) == "<VILLE_ETAPE>" and ville_etape:
            replace_and_label(match, ville_etape, "etape")

        pos = match.end()

    # Ajouter le reste de la phrase après la dernière balise
    if pos < len(template):
        remaining_words = template[pos:].split()
        tokens.extend(remaining_words)
        labels.extend(["O"] * len(remaining_words))

    return {"tokens": tokens, "labels": labels}

# Génération de 10,000 exemples labellisés
dataset_size = 2000
dataset = []

for _ in range(dataset_size):
    labelled_data = generate_phrase_and_labels(templates, villes)
    dataset.append(labelled_data)

# Sauvegarder le dataset dans un fichier CSV
output_file = "dataset_phrases_labelled_test.csv"
with open(output_file, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["tokens", "labels"])  # Noms des colonnes
    for item in dataset:
        writer.writerow([item['tokens'], item['labels']])

print(f"Dataset de {dataset_size} phrases labellisées généré et sauvegardé dans {output_file}.")

Dataset de 2000 phrases labellisées généré et sauvegardé dans dataset_phrases_labelled_test.csv.
